# Script Prep.

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.append('/content/drive/Shareddrives/NUTRECON/1. Acquisition/current_version/notebooks')

from nutrecon import *

import numpy as np
from glob import glob
from datetime import datetime
import json

from random import shuffle, sample
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import seaborn as sns

experiment_code = 'nutre'
dataPath = root[:-1] + "2. Data/raw/nutrecon/"
sequences_dataPath = dataPath + 'sequences/'
responses_dataPath = dataPath + 'responses/'

pres_order_d3_fileID = 'day3_presOrder'
assoc3_order_fileID = 'day3_atestOrder'
neuroEcon_responses_fileID = 'day3_neuroEcon'

neurEconRealization3_fileID = 'day3_neurEconRealization'


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# 1. Flavor tasting order 


In [5]:
subject_nr = 3    # Change subject number here

# ==============================================================================

subject_code = '{}{:03d}'.format(experiment_code, subject_nr)
presentation_order_d3_df, _ = loadResponses(sequences_dataPath, pres_order_d3_fileID, subject_code)
presentation_order_d3_df.T

,Flavor,image,flavor_id,image_id
0,cashew,square,c,Image I
1,ginseng,cross,i,Image VI


# 2. Association test order  (day 3)

In [6]:
associationTest3_df, _ = loadResponses(sequences_dataPath, assoc3_order_fileID, subject_code)
associationTest3_df.T

,Flavor,image,flavor_id,image_id
1,ginseng,cross,i,Image VI
2,cashew,square,c,Image I
3,ginseng,cross,i,Image VI
4,cashew,square,c,Image I
5,cashew,square,c,Image I
6,ginseng,cross,i,Image VI
7,ginseng,cross,i,Image VI
8,cashew,square,c,Image I
9,ginseng,cross,i,Image VI
10,ginseng,cross,i,Image VI


# 3. Reward realization (day 3)

In [7]:
df, responses_path = loadResponses(responses_dataPath, neuroEcon_responses_fileID, subject_code)

seed = int(responses_path.split('_')[-1].split('.')[0])
rng = np.random.default_rng(int(responses_path.split('_')[-1].split('.')[0]))

df_clean = df[df['choice'] != 0]

mix_yogurt_df = df_clean[df_clean['Trial Type'] == 'mixed_yogurt']
st_mt_df = df_clean[df_clean['Trial Type'] != 'mixed_yogurt']

realizationTrials_df  = pd.concat([mix_yogurt_df.sample(1), st_mt_df.groupby(['Trial Type', 'lottery type']).sample(1)]).reset_index(drop = True)

realizationResults_df = realizationTrials_df.apply(lambda row: realizeChoices(row,rng), axis = 1)
realizationResults_df = save_json(realizationResults_df, subject_code, neurEconRealization3_fileID , sequences_dataPath)
realizationResults_df[['Trial Type', 'reference type', 'lottery type', 'reference qt', 'lottery qt', 'reference p', 'lottery p', 'choice', 'reward Qt.',	'reward description']]


More than one file found for this subject. Type the number of the file you wish to select:
	0 -> /content/drive/Shareddrives/NUTRECON/2. Data/raw/nutrecon/responses/nutre003_day3_neuroEcon_1671012450.json saved on 23/11/2022 at 12:20:1669206000.
	1 -> /content/drive/Shareddrives/NUTRECON/2. Data/raw/nutrecon/responses/nutre003_day3_neuroEcon_1671012487.json saved on 23/11/2022 at 12:20:1669206000.
Invalid response. Type the number of the file you wish to select:
1


,Trial Type,reference type,lottery type,reference qt,lottery qt,reference p,lottery p,choice,reward Qt.,reward description
0,mixed_yogurt,C+,C-,80.0,80,0.5,0.50,1,0.0,You got nothing
1,mixed,money,C+,0.2,150,1.0,0.22,2,0.0,You got nothing
2,mixed,money,C-,0.2,150,1.0,0.75,2,150.0,mL of cashew yogurt
3,same,C+,C+,20.0,200,1.0,0.22,2,0.0,You got nothing
4,same,C-,C-,20.0,120,1.0,0.38,1,20.0,mL of cashew yogurt
5,same,money,money,1.0,1,1.0,0.50,1,1.0,€


___
___
___
___

# (in preparation...) 

In [ ]:
column_names = ['reference type', 'reference qt', 'reference p',
                'lottery type', 'lottery qt', 'lottery p',
                'ref_alpha', 'lott_alpha', 'beta',
                'ref_EU', 'lott_EU', 'pL', 'choice',
                'ref_alphaEstimates', 'lott_alphaEstimates', 'betaEstimates',
                'ref_alphaEstimStdErr', 'lott_alphaEstimStdErr', 'betaEstimStdErr']

optimize_cols = column_names[:6]  + [column_names[12]]
optimize_cols

['reference type',
 'reference qt',
 'reference p',
 'lottery type',
 'lottery qt',
 'lottery p',
 'choice']

In [ ]:
def get_EU_(p,X, alpha):
  return p * X**alpha

def get_pL_(euL, euR, beta):
  return 1 - 1/(1 + np.exp(beta * (euL - euR)))


def get_likelihood(row, params, cols = optimize_cols):
  ref_type = row[cols[0]]
  lott_type = row[cols[3]]
  if len(params) == 4:
    (alpha_money, alpha_Cplus, alpha_Cminus, beta) = params
    alphas = {'money' : alpha_money,
             'C+' : alpha_Cplus,
             'C-' : alpha_Cminus}
    ref_alpha = alphas[ref_type]
    lott_alpha = alphas[lott_type]
  elif len(params) == 6:
    (alpha_money, alpha_Cplus, alpha_Cminus, beta_money, beta_Cplus, beta_Cminus) = params
    alphas = {'money' : alpha_money,
             'C+' : alpha_Cplus,
             'C-' : alpha_Cminus}
    betas = {'money' : beta_money,
             'C+' : beta_Cplus,
             'C-' : beta_Cminus}
    ref_alpha = alphas[ref_type]
    lott_alpha = alphas[lott_type]
    beta = betas[lott_type]
  
  choice = row[cols[6]]

  ref_X = row[cols[1]]
  ref_p = row[cols[2]]
  ref_EU = get_EU_(ref_p, ref_X, ref_alpha)
  
  lott_X = row[cols[4]]
  lott_p = row[cols[5]]
  lott_EU = get_EU_(lott_p, lott_X, lott_alpha)

  pL = get_pL_(lott_EU, ref_EU, beta)
  if choice == 1:
    likelihood = pL
  else:
    likelihood = 1 - pL
  return likelihood

def get_negLogLikelihood(params, args):

  df = args
  task_cols = optimize_cols
  # compute likelihood of each choice
  likelihood = df.apply(lambda row: get_likelihood(row, params, cols = task_cols), axis=1).values
  # Take negative of logLikelihood for convention
  negloglikelihood = - np.sum(np.log(likelihood))
  return negloglikelihood

In [ ]:
# Import Packages
#import numpy as np
#import pandas as pd
#import seaborn as sns
#from random import randint
from scipy.optimize import minimize
#from sklearn.linear_model import LogisticRegression
from numpy.core.fromnumeric import resize
alphaMoney0 = 0
alphaCplus0 = 0
alphaCminus0 = 0
beta0 = 0

df_sameType = df_clean[df_clean['Trial Type'] == 'same']

args = (df_sameType)
x0 = (alphaMoney0, alphaCplus0, alphaCminus0, beta0)
res = minimize(get_negLogLikelihood, x0, args=args )

print('{}\n  - parameters: {}\n  - std. error: {}'.format(res.message, res.x, np.sqrt(np.diag(res.hess_inv))))
print('\nConfidene intervals:')
parsCI = ['{} \xb1 {}'.format(round(res.x[p],3), round(1.96*np.sqrt(np.diag(res.hess_inv))[p],3)) for p in range(len(res.x))]
for p in range(len(parsCI)): print('  - parameter {}: {}'.format(p + 1, parsCI[p]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log


Optimization terminated successfully.
  - parameters: [ 0.78404019  0.72824823 -0.69832033 -0.73646625]
  - std. error: [0.08001419 0.0343947  0.46020782 0.13769937]

Confidene intervals:
  - parameter 1: 0.784 ± 0.157
  - parameter 2: 0.728 ± 0.067
  - parameter 3: -0.698 ± 0.902
  - parameter 4: -0.736 ± 0.27


In [ ]:
# Import Packages
#import numpy as np
#import pandas as pd
#import seaborn as sns
#from random import randint
from scipy.optimize import minimize
#from sklearn.linear_model import LogisticRegression
from numpy.core.fromnumeric import resize
alphaMoney0 = 0
alphaCplus0 = 0
alphaCminus0 = 0
beta0 = 0

df, _ = loadResponses(responses_dataPath, 'day2_neuroEcon', subject_code)

df_clean = df[df['choice'] != 0]


df_sameType = df_clean[df_clean['Trial Type'] == 'same']

args = (df_sameType)
x0 = (alphaMoney0, alphaCplus0, alphaCminus0, beta0)
res = minimize(get_negLogLikelihood, x0, args=args )

print('{}\n  - parameters: {}\n  - std. error: {}'.format(res.message, res.x, np.sqrt(np.diag(res.hess_inv))))
print('\nConfidene intervals:')
parsCI = ['{} \xb1 {}'.format(round(res.x[p],3), round(1.96*np.sqrt(np.diag(res.hess_inv))[p],3)) for p in range(len(res.x))]
for p in range(len(parsCI)): print('  - parameter {}: {}'.format(p + 1, parsCI[p]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log


Optimization terminated successfully.
  - parameters: [ 0.47469489  0.3741134  -0.70411887 -2.00596335]
  - std. error: [0.04448418 0.03362573 0.2459837  0.27446021]

Confidene intervals:
  - parameter 1: 0.475 ± 0.087
  - parameter 2: 0.374 ± 0.066
  - parameter 3: -0.704 ± 0.482
  - parameter 4: -2.006 ± 0.538


com alphas negativos, EU diminui com o aumento da reward - faz sentido com o report da pessoa

quanto maior o valor absoluto do alpha, maior a diferença nas EU

In [ ]:
20**(-4.)

0.000625

In [ ]:
alpa = -.4
(10**alpa)/(100**alpa)

2.5118864315095797